In [52]:
import yfinance as yf
import os 
import pandas as pd 
import numpy as np
import csv



In [53]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
sp500_tickers = sp500_table['Symbol'].tolist()
sp500_tickers = ["AOS"]


In [ ]:
total_data = []
for ticker in sp500_tickers:
    try:
        df = yf.download(ticker, period="max", progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        if df.empty:
            continue

        df['Ticker'] = ticker
        df.reset_index(inplace=True)
        
        file = f"{ticker} Data.csv"
        df.to_csv(file, index=False)
        print(df.head())
        print(df.tail())
    except Exception as e:
        print(f"error, {e}")
